# Линейные слои

При работе с изображениями на данный момент мы вытягиваем их в вектор и обрабатываем так же как и обычные табличные признаки

![](./img/linear.png)

Проблема такого подхода кроется в том что мы можем подавать пиксели в любом порядке относительно друг друга и логика обучения/предсказания не будет менятся, наши веса всего лишь сменят индексы.

Что же делать?

# 2. Свертка

Давайте введем математическую операцию которая будет учитывать то как располагаются признаки относительно друг друга, скажем мы исследуем какой то процесс во времени и хотим обрабатывать временные признаки по 2 отсчета за раз.

число 2 назовем размером ядра свертки

![](./img/conv1d.png)

Тогда для вектора размером 5, у нас будет 4 шага вычислений, на шагу на каждую соседнюю пару и в результате получим вектор размера 4

Это уже больше похоже на то как устроены изображение, наш признак теперь зависит от того какие соседние пиксели попали в свертку, однако пока что мы решили проблему в рамках лево-право, а в изображении есть еще дополнительная размерность вверх-вниз

Что же делать?

# 2D свертка

Давайте будем обходить не вектор, а целую матрицу и наша свертка будет не вектором а матрицей, у ядра будет два параметра размер по горизонтали и размер по вертикали и будем обходить изображение слева направо и сверху вниз

![](./img/2d_convolution_1.png)

Итак мы имеем инструмент чтобы перевести пиксели изображения в карту признаков, однако мы работаем с одной матрицей, а стандартное изображение обычно идет в 3х каналах RGB

![](./img/conv.gif)

# Padding

Как мы можем заметить выходная карта признаков меньше чем исходное изображение, что не всегда хорошо иногда нам нужно сохранять размер карты признаков равной размеру изображения, тогда для этого мы можем добавлять дополнительные края к изображению - padding

Существует несколько типов Padding

1. Нулевой - добавляем края заполненные нулем
2. Зеркальный - отзеркаливаем относительно края
3. Краевой - используем соседние значения с краев

![](./img/padded.gif)

# Stride

Не всегда мы хотим просчитывать все соседние части изображения, иногда мы хотим сильно уменьшить размер карты признаков, для этого шаг делается больше 1

![](./img/strided.gif)

$$\large{Size_{new} = (Size_{old} - K + P) / S + 1}$$

# Dilation

И последнее изменение обхода связано с тем что расчитывать признак мы можем не хотеть не с соседних пикселей, а с пикселей который равноудаленны на какое то растояние - dilation

![](./img/delated.gif)

# Сумма сверток

Давайте для каждого цвета заведем свое отдельное ядро, рассчитаем карту признаков для каждого цвета и сложим эти карты между собой, прибавив свободный член, так же как и в линейном слое

![](./img/conv_channel.jpeg)

# Множество выходов

Теперь когда у нас есть параметры:

- размеры ядра
- количество входных каналов

Мы можем добавить дополнительный параметр количество выходных каналов, т.е. мы будем расчитывать множество карт признаков вместо одной

![](./img/filters.png)

# Poolings

Иногда для изображений не обязательно считать карту признаков через матрицу, а важно только максимальное/среднее значение, для этой операции существует слой называемый Max/AveragePooling

Он работает практически так же как и Convolutional слой, разница лишь в том что в ядре мы берем максимум/среднее и значения по разным каналам не суммируются, т.е. количество выходных каналов равно входным.

![](./img/max_pool.png)

In [2]:
import torch
import torch.nn as nn

In [3]:
conv_layer = nn.Conv2d(
    in_channels=3, # вход
    out_channels=16, # выход
    kernel_size=(3, 3), # размер ядра
    stride=(1, 1), # шаг
    dilation=(0, 0), # разреженность
    padding_mode="zeros", # вставка нулями
    padding=(1, 1), # размер отступа
    bias=True, # нужен ли свободный член
)

In [4]:
max_pool_layer = nn.MaxPool2d(
    kernel_size=(3, 3), # размер ядра
    stride=1, # шаг
    dilation=0, # разреженность
)

In [6]:
avg_pool_layer = nn.AvgPool2d(
    kernel_size=(3, 3), # размер ядра
    stride=1, # шаг
)

# Backward pass

Для сверточного слоя, обратный шаг так же будет сверткой

![](./img/back.png)

# AlexNet

В компьютерном зрении проводится основное соревнование, на классификацию изображений на датасете ImageNet

ImageNet - это огромный датасет с 1000 классами, задача состоит в том чтобы топ 5 классов предсказанных моделью с наибольшей вероятностью включали в себя истинный класс изображения.

До 2014 года качество ручной разметки превосходило все модели которые существовали, однако с появлением сверточных сетей, модель AlexNet смогла побить этот результат

![](./img/alexnet.png)